In [1]:
import numpy as np
import pandas as pd 
import copy
import os

import matplotlib.pyplot as plt
import seaborn as sns

from random import shuffle, randint

from numpy import mean, std, dstack

from pandas import read_csv

from keras import backend as K
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Model, Sequential
from keras.layers import (Input, Dense, Flatten, Dropout, Conv1D, LSTM, GRU,
                          TimeDistributed, GlobalAveragePooling1D, MaxPooling1D)
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.merge import concatenate

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler

from keras.optimizers import Adam, RMSprop,SGD
from keras.utils import to_categorical
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from scipy import signal

%matplotlib inline
with pd.option_context("display.max_rows", 10, "display.max_columns", 20):
    print(pd.get_option("display.max_rows")) 
    print(pd.get_option("display.max_columns"))
    
# Check GPU support 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.
/home/jurgen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jurgen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jurgen/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jurgen/.local/lib/python3.6/site-packages/tensorflow/python

10
20
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8526005461188308265
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2574739499674547786
physical_device_desc: "device: XLA_CPU device"
]


In [2]:
from os import chdir as cd
cd('..')

In [3]:
from seriem_temporis.display_functions import display_block_of_values
from seriem_temporis.controller import SignalController, KasperskySetSignalController
from neural_models.models import SplitConvolutionalAnomalyDetector

In [4]:
neural_model = SplitConvolutionalAnomalyDetector()

Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988, 500, 55)
(2988, 500, 2)
Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
neural_model.fit_model(verbose=1, epochs_per_step=5, batch_size=128)

Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988, 500, 55)
(2988, 500, 2)
Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988, 500, 55)
(2988, 500, 2)
Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988, 500, 55)
(2988, 500, 2)
Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988, 500, 55)
(2988, 500, 2)
Number of columns: 60
NaN Values: False
Number of columns after clean: 55
Successfully scaled control_results
savgol filter smoothing successful
(2988, 500, 1)
anomaly exist
(2988,

Initialise neural model

In [26]:
from sklearn.metrics import f1_score, precision_score, recall_score
def IOU_score(y_true, y_pred):
    IOU = []
    for idx, i in tf.map_fn(enumerate(y_pred)):
        OL = 0
        UN = 0
        for _idx, result in enumerate(i):
            if result ==1 and y_true[idx,_idx]==1:
                OL += 1  
            if result ==1 or y_true[idx,_idx]==1:
                UN += 1
        if OL != 0 and UN != 0:
            IOU.append(OL/UN)
    return mean(IOU)

### Estimate model results 

In [53]:
y_pred = model_cont.predict([X_test,X_test,X_test])
y_anomaly_test = Y_test[:,:,1]
y_anomaly_prediction = y_pred[:,:,1]

In [54]:
decision_threshold = 0.4
y_anomaly_prediction = (y_anomaly_prediction > decision_threshold).astype(int)

In [55]:
print(np.unique(y_anomaly_prediction, return_counts=True))
print(np.unique(y_anomaly_test, return_counts=True))
print(y_anomaly_prediction.shape)
print(y_anomaly_test.shape)

(array([0]), array([2934000]))
(array([0., 1.], dtype=float32), array([2933620,     380]))
(1956, 1500)
(1956, 1500)


In [42]:
cummulative_test = y_anomaly_test.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])
cummulative_test_pred = y_anomaly_prediction.reshape(y_anomaly_test.shape[0]*y_anomaly_test.shape[1])

In [43]:
print(cummulative_test.shape)
print(cummulative_test_pred.shape)

(2934000,)
(2934000,)


In [44]:
_f1_score = f1_score(cummulative_test, cummulative_test_pred)
_recall_score = recall_score(cummulative_test, cummulative_test_pred)
_precision_score = precision_score(cummulative_test, cummulative_test_pred)

/home/jurgen/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [45]:
print(_f1_score, _recall_score, _precision_score)

0.0 0.0 0.0


In [49]:
IOU = []
for idx, i in enumerate(y_anomaly_prediction):
    OL = 0
    UN = 0
    for _idx, result in enumerate(i):
        if result ==1 and y_anomaly_test[idx,_idx]==1:
            OL += 1  
        if result ==1 or y_anomaly_test[idx,_idx]==1:
            UN += 1
    if OL != 0 and UN != 0:
        IOU.append(OL/UN)

In [50]:
print(mean(IOU))

nan


/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/jurgen/.local/lib/python3.6/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
